In [1]:
from skfp.datasets.moleculenet import load_bace, load_clintox, load_bbbp, load_esol, load_lipophilicity, load_muv, load_pcba, load_tox21, load_toxcast
from skfp.model_selection import scaffold_train_test_split, butina_train_test_split, maxmin_train_test_split, randomized_scaffold_train_test_split
from skfp.fingerprints import ECFPFingerprint, MACCSFingerprint, RDKitFingerprint
from skfp.preprocessing import MolFromSmilesTransformer

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, make_union
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, r2_score

import numpy as np

/home/adrian/dev/UM-fingerprints/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipelines = {
    "baseline_pipeline": make_pipeline(MolFromSmilesTransformer(), make_union(ECFPFingerprint(count=True), MACCSFingerprint()), LogisticRegression(max_iter=1000)),
    "rf_pipeline": make_pipeline(MolFromSmilesTransformer(), make_union(ECFPFingerprint(count=True), MACCSFingerprint(), RDKitFingerprint()), RandomForestClassifier(random_state=41, class_weight='balanced')),
    "gb_pipeline": make_pipeline(MolFromSmilesTransformer(), make_union(ECFPFingerprint(count=True), MACCSFingerprint(), RDKitFingerprint()), GradientBoostingClassifier())
}

In [3]:
def calculate_scores(y_test, y_pred, y_proba):
    r2score = r2_score(y_test, y_pred)
    print(f"R2 score for Linear regression model: {r2score:.4f}")

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for Linear regression model: {accuracy:.4f}")

    roc_auc = roc_auc_score(y_test, y_proba[:, 1])
    print(f"ROC-AUC score for Linear regression model: {roc_auc:.4f}")

## Classification with the Logistic regression and RFC based on the clintox dataset; wihout filtering molecules

### Clintox dataset split

In [80]:
smiles, labels = load_bace()

In [81]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

1513
O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2ccccc2C)C)CC1(C)C 1
Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(=O)C)(CC(C)C)C1=O)CCc1ccccc1)[C@H](O)[C@@H]1[NH2+]C[C@H](OCCC)C1 1


FingerprintEstimatorGridSearch(estimator_cv=GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1),
                                                         param_grid={'min_samples_split': [2,
                                                                                           3,
                                                                                           4]}),
                               fingerprint=ECFPFingerprint(n_jobs=-1),
                               fp_param_grid={'radius': [2, 3]})

In [ ]:
fp_cv.predict(smiles)

In [52]:
#labels = [l[0] for l in labels]

In [53]:
smiles_train, smiles_test, y_train, y_test = scaffold_train_test_split(
    smiles, labels, test_size=0.2
)

### Checking the class distibution of the dataset 

In [54]:
print(f"Class distribution: {np.bincount(labels)}")

Class distribution: [822 691]


## Baseline with Logistic regression model

In [ ]:
pipelines['baseline_pipeline'].fit(smiles_train, y_train)
y_pred = pipelines['baseline_pipeline'].predict(smiles_test)
y_proba = pipelines['baseline_pipeline'].predict_proba(smiles_test) 

In [ ]:
calculate_scores(y_test, y_pred, y_proba)

R2 score for Linear regression model: -0.1929
Accuracy for Linear regression model: 0.7096
ROC-AUC score for Linear regression model: 0.7980


In [105]:
from skfp.datasets.moleculenet import load_bace
from skfp.fingerprints import ECFPFingerprint
from skfp.model_selection import FingerprintEstimatorGridSearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

smiles, labels = load_bace()

fp = ECFPFingerprint(n_jobs=-1)
fp_params = {"radius": [2, 3]}
clf = RandomForestClassifier(n_jobs=-1)
clf_params = {"min_samples_split": [2, 3, 4]}
clf_cv = GridSearchCV(clf, clf_params)
fp_cv = FingerprintEstimatorGridSearch(fp, fp_params, clf_cv)

In [106]:
smiles_train, smiles_test, y_train, y_test = scaffold_train_test_split(
    smiles, labels, test_size=0.2
)

In [107]:
fp_cv.fit(smiles_train, y_train)
y_pred = fp_cv.predict(smiles_test)
y_proba = fp_cv.predict_proba(smiles_test) 

In [108]:
calculate_scores(y_test, y_pred, y_proba)

R2 score for Linear regression model: -0.1794
Accuracy for Linear regression model: 0.7129
ROC-AUC score for Linear regression model: 0.7921


## RF classifier

In [ ]:
pipelines['rf_pipeline'].fit(smiles_train, y_train)
y_pred = pipelines['rf_pipeline'].predict(smiles_test)
y_proba = pipelines['rf_pipeline'].predict_proba(smiles_test)  

In [ ]:
calculate_scores(y_test, y_pred, y_proba)

R2 score for RF classifier: -0.3556
Accuracy for RF classifier: 0.6700
ROC-AUC score for RF classifier: 0.7763


In [45]:
# filtering molecules from the test dataset 

from skfp.filters import PfizerFilter

filt = PfizerFilter(allow_one_violation=True)

In [46]:
smiles_to_label = dict(zip(smiles_train, y_train))

In [47]:
filtered_smiles = filt.transform(smiles_train)
filtered_labels = [smiles_to_label[smi] for smi in filtered_smiles]

In [ ]:
pipelines['rf_pipeline'].fit(filtered_smiles, filtered_labels)

Pipeline(steps=[('molfromsmilestransformer', MolFromSmilesTransformer()),
                ('featureunion',
                 FeatureUnion(transformer_list=[('ecfpfingerprint',
                                                 ECFPFingerprint(count=True)),
                                                ('maccsfingerprint',
                                                 MACCSFingerprint()),
                                                ('rdkitfingerprint',
                                                 RDKitFingerprint())])),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced',
                                        random_state=0))])

In [ ]:
y_pred = pipelines['rf_pipeline'].predict(smiles_test)
y_proba = pipelines['rf_pipeline'].predict_proba(smiles_test)

In [ ]:
calculate_scores(y_test, y_pred, y_proba)

R2 score for RF classifier: -0.4234
Accuracy for RF classifier: 0.6535
ROC-AUC score for RF classifier: 0.7345


## CLINTOX

In [78]:
smiles, labels = load_clintox()

In [79]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

1477
[C@@H]1([C@@H]([C@@H]([C@H]([C@@H]([C@@H]1Cl)Cl)Cl)Cl)Cl)Cl [1 0]
[C@H]([C@@H]([C@@H](C(=O)[O-])O)O)([C@H](C(=O)[O-])O)O [1 0]


In [ ]:
pipelines['baseline_pipeline'].fit(smiles_train, y_train)
y_pred = pipelines['baseline_pipeline'].predict(smiles_test)
y_proba = pipelines['baseline_pipeline'].predict_proba(smiles_test) 

## BBBP

In [92]:
smiles, labels = load_bbbp()

In [93]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

2039
[Cl].CC(C)NCC(O)COc1cccc2ccccc12 1
C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl 1


In [ ]:
smiles_train, smiles_test, y_train, y_test = scaffold_train_test_split(
    smiles, labels, test_size=0.2
)

NameError: name 'train_test_split' is not defined

In [95]:
print(f"Class distribution: {np.bincount(labels)}")

Class distribution: [ 479 1560]


In [96]:
pipelines['baseline_pipeline'].fit(smiles_train, y_train)
y_pred = pipelines['baseline_pipeline'].predict(smiles_test)
y_proba = pipelines['baseline_pipeline'].predict_proba(smiles_test) 

[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not removing hydrogen atom without neighbors
[03:04:09] WARNING: not r

In [97]:
calculate_scores(y_test, y_pred, y_proba)

R2 score for Linear regression model: 0.0241
Accuracy for Linear regression model: 0.7574
ROC-AUC score for Linear regression model: 0.8362


## ESOL

In [66]:
smiles, labels = load_esol()

In [67]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

1128
OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O  -0.77
Cc1occc1C(=O)Nc2ccccc2 -3.3


## LIPOP

In [68]:
smiles, labels = load_lipophilicity()

In [69]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

4200
Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14 3.54
COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)CCc3ccccc23 -1.18


## MUV

In [70]:
smiles, labels = load_muv()

In [71]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

93087
Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C [nan nan nan nan nan nan nan  0. nan nan nan  0. nan nan nan nan nan]
Cn1ccnc1SCC(=O)Nc1ccc(Oc2ccccc2)cc1 [ 0.  0. nan nan  0.  0.  0. nan nan nan  0. nan  0. nan nan  0.  0.]


## PCBA

In [72]:
smiles, labels = load_pcba()

In [73]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

437929
CC(=O)N1CCC2(CC1)NC(=O)N(c1ccccc1)N2 [ 0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.
  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0. nan  0.  0.  0. nan  0. nan  0.  0. nan  0.  0.  0.
  0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.  0.  0. nan  0.  0.
  0.  0.  1. nan  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
N#Cc1nnn(-c2ccc(Cl)cc2)c1N [ 0.  0. nan  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. nan  0.  0.  0. na

## TOX21

In [74]:
smiles, labels = load_tox21()

In [75]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

7831
CCOc1ccc2nc(S(N)(=O)=O)sc2c1 [ 0.  0.  1. nan nan  0.  0.  1.  0.  0.  0.  0.]
CCN1C(=O)NC(c2ccccc2)C1=O [ 0.  0.  0.  0.  0.  0.  0. nan  0. nan  0.  0.]


## TOXCAST

In [76]:
smiles, labels = load_toxcast()

In [77]:
print(len(smiles))
print(smiles[0], labels[0])
print(smiles[1], labels[1])

8576
[O-][N+](=O)C1=CC=C(Cl)C=C1 [ 0.  0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  